# Validation of RWKV v5 model inference code

In [2]:
# Update the RWKV pip package, found here : https://pypi.org/project/rwkv/
!python3 -m pip install --upgrade rwkv

In [2]:
INFERENCE_MODE="cpu"
INFERENCE_TYPE="fp32"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

NOTEBOOK_DIR: /home/picocreator/rwkv-proj/infctx-dev/notebook/trainer-v5-validation
TRAINER_DIR: /home/picocreator/rwkv-proj/infctx-dev/RWKV-v5
PROJECT_DIR: /home/picocreator/rwkv-proj/infctx-dev


In [13]:
!mkdir -p ../../model/
!cd ../../model/ && wget -nc "https://huggingface.co/BlinkDL/rwkv-5-world/resolve/8eb0273bd6935fa310c57532637d93d055d72f05/RWKV-5-World-1B5-v2-20231025-ctx4096.pth"
!cd ../../model/ && ls
!cd ../../model/ && pwd

File ‘RWKV-5-World-1B5-v2-20231025-ctx4096.pth’ already there; not retrieving.

Echo-A-1B5-Init.pth	 L32-D2560-neox-init.pth
L24-D2048-init.pth	 L32-D4096-neox-init.pth
L24-D2048-neox-init.pth  RWKV-5-World-1B5-v2-20231025-ctx4096.pth
/home/picocreator/rwkv-proj/infctx-dev/model


## Reference code inference

In [9]:
import os
os.environ['RWKV_JIT_ON'] = '1'
os.environ["RWKV_CUDA_ON"] = '0' # '1' to compile CUDA kernel (10x faster), requires c++ compiler & cuda libraries

from rwkv.model import RWKV
from rwkv.utils import PIPELINE, PIPELINE_ARGS

model = RWKV(model=os.path.join(PROJECT_DIR, "model/RWKV-5-World-1B5-v2-20231025-ctx4096.pth"), strategy='cpu fp32')
pipeline = PIPELINE(model, "rwkv_vocab_v20230424") # Using the world tokenizer

ctx = "\nIn a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese."
print(ctx, end='')

def my_print(s):
    print(s, end='', flush=True)

# # For alpha_frequency and alpha_presence, see "Frequency and presence penalties":
# # https://platform.openai.com/docs/api-reference/parameter-details

# print("\n")

# We use top_k = 1 to effectively always take the highest choice token
args = PIPELINE_ARGS(temperature = 0.5, top_p = 0.7, top_k = 1, # top_k = 0 then ignore
                    #  alpha_frequency = 0.25,
                    #  alpha_presence = 0.25,
                    #  alpha_decay = 0.996, # gradually decay the penalty
                     token_ban = [0], # ban the generation of some tokens
                     token_stop = [], # stop generation whenever you see any token here
                     chunk_len = 256) # split input into chunks to save VRAM (shorter -> slower)

pipeline.generate(ctx, token_count=200, args=args, callback=my_print)
print('\n')

# out, state = model.forward([187, 510, 1563, 310, 247], None)
# print(out.detach().cpu().numpy())                   # get logits
# out, state = model.forward([187, 510], None)
# out, state = model.forward([1563], state)           # RNN has state (use deepcopy to clone states)
# out, state = model.forward([310, 247], state)
# print(out.detach().cpu().numpy())                   # same result as above
# print('\n')

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading /home/picocreator/rwkv-proj/infctx-dev/model/RWKV-5-World-1B5-v2-20231025-ctx4096.pth ...


Strategy: (total 24+1=25 layers)
* cpu [float32, float32], store 25 layers
0-cpu-float32-float32 1-cpu-float32-float32 2-cpu-float32-float32 3-cpu-float32-float32 4-cpu-float32-float32 5-cpu-float32-float32 6-cpu-float32-float32 7-cpu-float32-float32 8-cpu-float32-float32 9-cpu-float32-float32 10-cpu-float32-float32 11-cpu-float32-float32 12-cpu-float32-float32 13-cpu-float32-float32 14-cpu-float32-float32 15-cpu-float32-float32 16-cpu-float32-float32 17-cpu-float32-float32 18-cpu-float32-float32 19-cpu-float32-float32 20-cpu-float32-float32 21-cpu-float32-float32 22-cpu-float32-float32 23-cpu-float32-float32 24-cpu-float32-float32 
emb.weight                        f32      cpu  65536  2048 
blocks.0.ln1.weight               f32      cpu   2048       
blocks.0.ln1.bias                 f32      cpu   2048       
blocks.0.ln2.weight               f32      cpu   2048       
blocks.0.ln2.bias                 f32      cpu   2048       
blocks.0.att.time_mix_k           f32      cpu   2048 

# Expected result should be

```

The researchers, who were led by Dr. David Doubilet, a photographer and filmmaker, were able to capture the dragons in their natural habitat. The team was able to film the dragons for over two hours, and they were able to capture the dragons in their natural habitat.
The researchers were able to capture the dragons in their natural habitat. The team was able to film the dragons for over two hours, and they were able to capture the dragons in their natural habitat.
The researchers were able to capture the dragons in their natural habitat. The team was able to film the dragons for over two hours, and they were able to capture the dragons in their natural habitat.
The researchers were able to capture the dragons in their natural habitat. The team was able to film the dragons for over two hours, and they were able to capture the dragons in their natural habitat.
The researchers were able to capture the dragons in their natural habitat. The team was able to film the dragons for
```


# RWKV infctx trainer, in inference mode

Should match the above result (200 tokens)

In [8]:
# Run the reference implementation
!cd $TRAINER_DIR && python3 ./dragon_test.py "../model/RWKV-5-World-1B5-v2-20231025-ctx4096.pth" "ref"

[2023-10-28 13:25:29,225] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
Traceback (most recent call last):
  File "/home/picocreator/rwkv-proj/infctx-dev/RWKV-v5/./dragon_test.py", line 52, in <module>
    model = SimpleRWKV(MODEL_PATH, device=DEVICE)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/picocreator/rwkv-proj/infctx-dev/RWKV-v5/src/model.py", line 1420, in __init__
    self.model = RWKV(**model_config)
                 ^^^^^^^^^^^^^^^^^^^^
  File "/home/picocreator/rwkv-proj/infctx-dev/RWKV-v5/src/model.py", line 670, in __init__
    self.load_state_dict(model_weights)
  File "/home/picocreator/anaconda3/envs/rwkv-exp/lib/python3.11/site-packages/torch/nn/modules/module.py", line 2041, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
RuntimeError: Error(s) in loading state_dict for RWKV:
	size mis